In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

# Import loaders (used by viewer and overlays)
from loader import (
    load_cgm_data,
    load_sleep_data,
    load_food_entry_data
)

# Import the CGMViewer class
from viewer import CGMViewer

# Import overlays on CGM (glucose) time-series
from overlays import (
    TimeInRangeOverlay,
    FoodEntryOverlay,
    MeanGlucoseOverlay,
)

# Import extensions (e.g. hypnogram) that will be attached in separate panel
from extensions import (
    HypnogramExtension
)

# Import CGM-based biomarker overlays
from overlays import (
    CgmMeasuresOverlay,
    CvOverlay,
    MageOverlay
)

# Import Multi-modal biomarker overlays
from overlays import (
    WakeupGlucoseOverlay,
    PPGROverlay
)

ImportError: cannot import name 'summarize_measures' from 'cgmquantify' (/opt/conda/lib/python3.13/site-packages/cgmquantify/__init__.py)

In [ ]:
# Local path to data if JHE client fails
base_path = "./sample_subject"

## Loading Data from the JupyterHealth Exchange

In [ ]:
# Initialize the JupyterHealth Client
# See documentation for more detail: https://jupyterhealth.org/software-documentation/
from jupyterhealth_client import JupyterHealthClient, Code
jh_client = JupyterHealthClient()

# Sample study and patient for the purpose of this tutorial
patient_id = 40079
study_id = 30017

# Get basic information about the patient
jh_client.get_patient(patient_id)

### Loading CGM Data from JHE

In [ ]:
# Using the client
client_cgm_df = jh_client.list_observations_df(
    patient_id = patient_id,
    study_id = study_id,
    code = Code.BLOOD_GLUCOSE.value,
    limit = 10_000
)

cgm_df = load_cgm_data(source="client", client_df=client_cgm_df)
cgm_df.head()

In [ ]:
# You can also load using the local file
cgm_df = load_cgm_data(source="file", base_path=base_path, filename="blood_glucose.json")
cgm_df.head()

### Loading Sleep Data

In [ ]:
client_sleep_df = jh_client.list_observations_df(
    patient_id = patient_id,
    study_id = study_id,
    code = Code.SLEEP_STAGE_SUMMARY.value
)

sleep_df = load_sleep_data(source="client", client_df=client_sleep_df)
sleep_df.head()

In [ ]:
sleep_df = load_sleep_data(source="file", base_path=base_path, filename="sleep_stage_summary.json")
sleep_df.head()

### Loading Food Entry Data

In [ ]:
client_food_df = jh_client.list_observations_df(
    patient_id = patient_id,
    study_id = study_id,
    code = Code.FOOD_ENTRY.value
)

food_df = load_food_entry_data(source="client", client_df=client_food_df)
food_df.head()

In [ ]:
food_df = load_food_entry_data(source="file", base_path=base_path, filename="food_entry.json")
food_df.head()

## Using the CGMSandbox

In [ ]:
### Initialize the CGMSandbox Viewer and draw daily glucose tracings
viewer = CGMViewer(source="client", 
                   client_df=client_cgm_df,
                   gl_range=(0, 250))

### Let's try to add some overlay's to the plain CGM tracing! 
viewer.add_overlay(TimeInRangeOverlay())
viewer.add_overlay(FoodEntryOverlay(source="client", client_df=client_food_df))

### Now, let us extend the plain CGM tracing by adding an hypnogram extension
viewer.add_extensions(HypnogramExtension(source="client", client_df=client_sleep_df))

### Let us use overlay to visualize our CGM-based biomarkers
viewer.add_overlay(CgmMeasuresOverlay())
viewer.add_overlay(CvOverlay())
viewer.add_overlay(MageOverlay(show_ma=True))

### We can utilize sleep and diet data to extract other meaninful markers
viewer.add_overlay(WakeupGlucoseOverlay(source="client", client_df=client_sleep_df))
viewer.add_overlay(PPGROverlay(source="client", client_df=client_food_df))

viewer.add_overlay(MeanGlucoseOverlay(source="client", scope="daily", client_df=client_cgm_df))

viewer.show()

In [ ]:
### You can create the same Viewer using local files
viewer = CGMViewer(source="file", 
                   base_path=base_path, 
                   filename="blood_glucose.json",
                   gl_range=(0, 250))

viewer.add_overlay(FoodEntryOverlay(source="file", 
                                    base_path=base_path,
                                    filename="food_entry.json"))

viewer.show()